## Import các thư viện cần thiết

In [1]:
# Import các thư viện cần thiết
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

D:\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Chuẩn bị dữ liệu

In [2]:
# Đọc dữ liệu từ file CSV
data = pd.read_csv('./Data/air_quality_cleaned.csv')

# Xem mẫu dữ liệu
data.head()

,dt,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2021-01-01 00:00:00,3,700.95,0.44,35.99,17.35,32.90,20.33,26.64,8.99
1,2021-01-01 01:00:00,3,847.82,2.46,38.04,18.06,36.24,23.32,30.54,9.37
2,2021-01-01 02:00:00,3,894.55,5.25,38.39,23.25,41.01,24.16,31.93,9.25
3,2021-01-01 03:00:00,3,827.79,6.20,36.33,33.98,43.39,23.20,30.91,8.61
4,2021-01-01 04:00:00,2,660.90,3.69,29.13,54.36,35.76,19.50,25.60,6.21


In [3]:
# Kiểm tra thông tin cơ bản về dữ liệu
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25176 entries, 0 to 25175
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   dt      25176 non-null  object 
 1   aqi     25176 non-null  int64  
 2   co      25176 non-null  float64
 3   no      25176 non-null  float64
 4   no2     25176 non-null  float64
 5   o3      25176 non-null  float64
 6   so2     25176 non-null  float64
 7   pm2_5   25176 non-null  float64
 8   pm10    25176 non-null  float64
 9   nh3     25176 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.9+ MB


## Huấn luyện mô hình

**Lựa chọn thuộc tính:** chúng em sẽ tiến hành phân loại chất lượng không khí theo chỉ số `aqi` dựa trên các chỉ số `no, co, so2, no2, o3, pm2_5, pm10, nh3`

In [4]:
# Chia dữ liệu thành features (X) và target variable (y)
X = data[['no', 'co', 'so2', 'no2', 'o3', 'pm2_5', 'pm10', 'nh3']]  # Chọn các chỉ số khí quyển làm features
y = data['aqi']  # Chọn cột AQI làm target variable

**Feature Scaling:**

- Khi khoảng giá trị giữa 2 thuộc tính quá cách xa nhau thì việc mô hình hóa cũng như trực quan mối quan hệ có thể gặp khó khăn, do đó phải thực hiện kĩ thuật 'Feature Scaling' hay việt hóa là 'Co giãn thuộc tính'.
- Có 3 phương pháp feature scaling chính là:
    - Standardisation (Chính quy hóa): Làm cho tập dữ liệu có trung bình là 0 và độ lệch chuẩn là 1 và được áp dụng cho hầu hết các trường hợp cần feature scaling.
    - Normalisation (Tiêu chuẩn hóa): Làm cho các giá trị trong tập dữ liệu thuộc đoạn [0, 1] và được áp dụng nếu tập dữ liệu tuân theo phân phối chuẩn.
    - MinMax Scaler: Đưa các giá trị về khoảng giữa 2 giá trị min và max trong miền giá trị của thuộc tính, có thể là đoạn [-1, 0], [0, 1], [-1, 1],...
- Trong bài này nhóm chọn phương pháp Standardisation để scaling khoảng giá trị của thuộc tính về khoảng gần hơn với giá trị của tập y là `aqi`.

In [5]:
# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X = scaler.fit_transform(X)

**Phân tách bộ dữ liệu thành 3 tập training set, validation set và test set:**

- Mục đích:
    - Huấn luyện và đánh giá độc lập: Tập huấn luyện được sử dụng để huấn luyện mô hình. Tập kiểm tra được sử dụng để đánh giá hiệu suất tổng quát của mô hình. Dữ liệu này không được sử dụng trong quá trình huấn luyện để đảm bảo mô hình có khả năng tổng quát hóa với dữ liệu mới, đảm bảo tính khách quan.
    - Kiểm tra: Tập validation được sử dụng để tìm ra mô hình tốt nhất với những siêu tham số khác nhau.
    - Tránh overfitting: Nếu không chia dữ liệu mà sử dụng toàn bộ dữ liệu để huấn luyện, mô hình có thể học "quá khớp" (overfit) dữ liệu huấn luyện mà không tổng quát hóa được cho dữ liệu mới. Việc có tập kiểm tra giúp đánh giá xem mô hình có đang học từ dữ liệu một cách tổng quát hay chỉ học "nhớ" dữ liệu huấn luyện không.
    - Đánh giá và cải thiện mô hình: Thông qua việc đánh giá trên tập kiểm tra, chúng ta có thể đánh giá hiệu suất của mô hình và cải thiện nó thông qua việc điều chỉnh các tham số hoặc phương pháp huấn luyện.
- Kích thước mỗi tập như sau:
    - Size of Training set = 80% * (Size of Dataset)
    - Size of Test set = 20% * (Size of Dataset).

In [6]:
# Chia dữ liệu thành tập train và tập test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tiếp tục chia tập train thành tập train và tập validation
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

### 1. Mô hình Decision Tree

- Mô hình Decision Tree (cây quyết định) là một thuật toán học máy có cấu trúc dạng cây, chia dữ liệu dựa trên các quy tắc quyết định đơn giản. Mỗi nút lá của cây đại diện cho một nhãn hoặc một giá trị dự đoán, trong khi các nút gốc và nội bộ biểu diễn các quy tắc quyết định để chia tách dữ liệu.

- Cách hoạt động chính của Decision Tree:

    - Chọn thuộc tính quan trọng: Các thuộc tính được chọn dựa trên độ quan trọng của chúng trong việc chia dữ liệu.
    - Tách nút (node splitting): Mỗi nút trong cây đại diện cho một thuộc tính và một ngưỡng (threshold). Dữ liệu được chia thành các nhánh dựa trên giá trị của thuộc tính này.
    - Xây dựng cây: Quá trình tách nút được thực hiện đệ quy cho đến khi một điều kiện dừng được đáp ứng (như đạt đến độ sâu tối đa hoặc không thể chia tách thêm).

**Tiến hành huấn luyện:**

- Đầu tiên, Nhóm tiến hành mô hình hóa dữ liệu theo *Decision Tree* với một cây khá đơn giản dựa theo thuật toán đo lường *Gini*, *độ sâu tối đa của cây* là 3, *số mẫu tối thiểu cần thiết để chia một node* là 2 và *số mẫu tối thiểu cần thiết ở mỗi leaf* là 1. 
- Sau đó đánh giá mô hình này dựa trên *Cross-validation* bằng cách chia tập train thành 5 phần bằng nhau và sử dụng 4 phần để huấn luyện và phần còn lại để đánh giá mô hình, quá trình này được lặp lại 5 lần, mỗi lần sử dụng một fold khác nhau làm tập kiểm tra. Điều này nhằm thử nghiệm tính hiệu quả của cây đơn giản này với bộ dữ liệu mà chúng em có.  

In [8]:
# Khởi tạo mô hình Decision Tree
model = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_split=2, min_samples_leaf=1)

# Huấn luyện mô hình trên tập train
model.fit(X_train, y_train)

# Đánh giá mô hình trên tập validation
#val_accuracy = model.score(X_val, y_val)
#print(f'Validation Accuracy: {val_accuracy}')

# Đánh giá mô hình bằng cross-validation trên tập train
cv_scores = cross_val_score(model, X_train, y_train, cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

# Đánh giá mô hình trên tập test (đánh giá cuối cùng)
test_accuracy = model.score(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')

Cross-Validation Scores: [0.80635551 0.81330685 0.80685204 0.80983118 0.80362463]
Mean CV Accuracy: 0.8079940417080437
Test Accuracy: 0.8105639396346307


**Đánh giá:**

- Có thể thấy việc chia tập train thành 4 fold và sử dụng Cross-Validation cho ra hiệu quả khá tương đồng nhau (Trung bình xấp xỉ 80%). Hiệu quả trên tập test ban đầu cũng khá tương tự như vậy (Xấp xỉ 81%). Kết quả này là khá cao, như vậy ta có thể thấy mô hình này khá hiệu quả.

**Cải thiện mô hình:**
- Mô hình có thể cho ra những kết quả tốt hơn với những tham số khác, vì vậy ta có thể kiểm tra thông qua thư viện `GridSearchCV`

In [10]:
"""
    Sử dụng cách chia tập train và tập test lần lượt là 80% và 20% của dataset, sau đó cũng sử dụng Cross-validation chia tập train thành 4 fold như trên
"""

# Chia dữ liệu thành tập train và tập test
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo mô hình Decision Tree
dtree = DecisionTreeClassifier()

# Định nghĩa lưới các giá trị tham số cần tìm kiếm
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 10, 15, 20],
    'min_samples_split': [2, 3, 5, 7, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5]
}

# Tạo đối tượng GridSearchCV
grid_search = GridSearchCV(dtree, param_grid, cv=5, scoring='accuracy')

# Thực hiện tìm kiếm trên lưới tham số
grid_search.fit(X_train, y_train)

# In ra tham số tốt nhất
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}


Như vậy ta đã có thông số params tốt nhất có thể tìm được trong tập các params đưa ra như trên, ta sẽ lưu lại thành `best_model` và đánh giá nó

In [11]:
# Đánh giá hiệu suất của mô hình tốt nhất trên tập kiểm tra
best_model_tree = grid_search.best_estimator_
y_pred = best_model_tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.8387609213661636


**Dự đoán:**
- Tiến hành dự đoán từ 1 sample data với `best_model` vừa tạo ra như trên

In [12]:
# Tạo sample data mới
new_data = {
    'no': 0.5,
    'co': 700.3,
    'no2': 37.3,
    'o3': 52.7,
    'so2': 35.1,
    'pm2_5': 19.5,
    'pm10': 20.2,
    'nh3': 7.99
}

# Chuyển sample data thành Dataframe
new_df = pd.DataFrame([new_data])

# Chuẩn hóa sample data
new_sample = scaler.transform(new_df)

# Dữ đoán aqi sử dụng best_model vừa huấn luyện trên 
predicted_aqi = best_model_tree.predict(new_sample)

print(f'Predicted AQI: {predicted_aqi}')

Predicted AQI: [2]


D:\Anaconda\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
